In [1]:
from azureml.core import Workspace

c:\Users\Vikas.Yetintala\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


# creating workspace

In [ ]:
ws= Workspace.create(name='Azureml-SDK-brillio',
                     subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad",
                     resource_group="RG-MSFT-ML-POC-01",
                     create_resource_group=True,
                     location="")
ws.write_config(path="./config")

In [ ]:
ws=Workspace.from_config("./config")

# creating a Datastore

In [ ]:
from azureml.core import Datastore
az_store = Datastore.register_azure_blob_container(workspace=ws,
                                                    datastore_name="azure_sdk_blob01",
                                                    account_name="vikaspoctitanic",
                                                    container_name="msbrilliopoc",
                                                    account_key="be/0iIZOayOkNh4ouG4Z6S4JR7fgVM9+R2bpLRcMwO3Z5mne0GEas/ZUNLDldRYqo4/oPDcatLge+AStdxl7hQ==")

In [ ]:
az_store= Datastore.get(ws,"azure_sdk_blob01")

# creating a Dataset

In [ ]:
from azureml.core import Dataset
csv_path= [(az_store,"titanic.csv")]
dataset= Dataset.Tabular.from_delimited_files(path=csv_path)
#register the dataset
dataset = dataset.register(workspace=ws,
                            name="Titanic_vikas",
                            create_new_version=True)

In [ ]:
ws_list=Workspace.list(subscription_id="a4df42c9-c1d2-44c4-b2c2-05edb4b27aad")
ws_list=list(ws_list)
az_default_store= ws.get_default_datastore()
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
ds_list=list(ws.datasets.keys())
for items in ds_list:
    print(items)

In [ ]:
ws=Workspace.from_config("./config")
az_store= Datastore.get(ws,"azure_sdk_blob01")
az_dataset=Dataset.get_by_name(ws,"Titanic_vikas")
az_default_store= ws.get_default_datastore()

In [ ]:
az_ds_from_df = Dataset.Tabular.register_pandas_dataframe(dataframe=df_sub,
                                                          target=az_store,
                                                          name="Titanic_selected_columns")

In [ ]:
files_list=["./data/test.csv","./data/test1.csv"]
files_list= az_store.upload_files(files=files_list,
                                   target_path="folder_name/",
                                relative_root="./data/",
                                overwrite=True)

In [ ]:
az_store.upload(src_dir="./data",
                target_path="folder_name/data",
                overwrite=True)

# Creating Experiment

In [ ]:
from azureml.core import Experiment
experiment=Experiment(workspace=ws,name="azureml_exp1_vikas")

# Running experiment by logging simple metrics

In [ ]:
new_run= experiment.start_logging()
total_observations=len(df)
null_df=df.isnull().sum()
new_run.log("Total observations: ",total_observations)
for columns in df.columns:
    new_run.log(columns,nulldf[columns])
new_run.complete()

# Running Experiment from Local with script file

In [ ]:
from azureml.core import Workspace, Experiment, ScriptRunConfig
new_experiment=Experiment(workspace=ws,name="azureml_exp2")
script_config= ScriptRunConfig(source_directory=".",
                               script="basicscript.py")
new_run = new_experiment.submit(config=script_config)
new_run.wait_for_completion()

# Creating Environment and attach to configuration

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
myenv= Environment(name="MyEnvironment")
myenv_dep= CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-explain-model','interpret-community'])
myenv.python.conda_dependencies = myenv_dep
myenv.register(ws)

In [ ]:
script_config = ScriptRunConfig(source_directory=".",
                                script="TrainingScript.py",
                                environment=myenv) 
new_run1 = new_experiment.submit(config=script_config)
new_run1.wait_for_completion()

# Creating Compute cluster in azure

In [ ]:
from azureml.core.compute import AmlCompute 
cluster_name= "comput1"
if cluster_name not in ws.compute_targets:
    compute_config= AmlCompute.provisioning_configuration(vm_size="",
                                                        max_nodes=2)
    cluster = AmlCompute.create(ws, cluster_name, compute_config)
    cluster.wait_for_completion()
else:
    cluster= ws.compute_targets[cluster_name]
print(cluster," compute cluster found.. using it....")

# Running Experiment in azure

In [ ]:
from azureml.train.automl import AutoMLConfig
automl_config= AutoMLConfig(task='classification',
                            compute_target=cluster,
                            training_data=input_ds,
                            validation_size=0.3,
                            label_column_name="",
                            primary_metric="norm_macro_recall",
                            iterations=10,
                            max_concurrent_iterations=2,
                            experiment_timeout_hours=0.25,
                            featurization='auto')

In [ ]:
from azureml.core.experiment import Experiment
new_exp= Experiment(ws,"exp_name")
print("submitting the exxperiment")
new_run= new_exp.submit(automl_config)
new_run.wait_for_completion(show_output=True)

In [ ]:
best_child_run= new_run.get_best_child()
print(best_child_run)

In [ ]:
for run in new_run.get_children():
    print("run id: ",run.id)
    print("accuracy is: ",run.get_metrics['accuracy'])
    print("norm macro recall: ",run.get_metrics['norm_macro_recall'])